In [1]:
# Imports
import time
import argparse
import pyspark
import numpy as np 
import pandas as pd
import datetime as dt
from pathlib import Path
from pyspark import SparkContext
from pyspark.sql.session import SparkSession

In [2]:
# Creating SparkSession
sparkSession = SparkSession(sc)

In [4]:
# Loading Data
file = 'dataset.csv'
dataset = pd.read_csv(file)
dataset.head()

In [8]:
# Shape
dataset.shape

(686327, 9)

In [10]:
# Datatypes
dataset.dtypes

Genero_Usuario     object
Idade_Usuario       int64
Bike                int64
Estacao_Aluguel     int64
Data_Aluguel       object
Hora_Aluguel       object
Estacao_Chegada     int64
Data_Chegada       object
Hora_Chegada       object
dtype: object

In [5]:
# To get data
def get_data(date, time):
    from datetime import datetime    
    datetime_object = datetime.strptime(date + " " + time, '%d/%m/%Y %H:%M:%S')
    return datetime_object

In [6]:
# to classify age
def classify_age(age):
    if age < 18:
        return "00-18"
    elif age >= 18 and age <= 34:
        return "18-34"
    elif age >= 35 and age <= 44:
        return "35-44"
    elif age >= 45 and age <= 54:
        return "45-54"
    elif age >= 55 and age <= 64:
        return "55-64"
    else:
        return "65+"

In [22]:
# Cleaning data
def cleaning_data(part_id, list_of_records):
    if part_id == 0: 
        next(list_of_records) 
    import csv
    reader = csv.reader(list_of_records)
    for row in reader:
        gender = row[0]
        age = int(row[1])
        bike_id = int(row[2])
        station_start = row[3]
        datetime_start = get_data(row[4], row[5])
        datetime_end = get_data(row[7], row[8])
        station_end = row[6]
        yield (bike_id, gender, classify_age(age), station_start, station_end, datetime_start, datetime_end)

In [19]:
# Creating RDD
def create_rdd(sc, input_file):
    print("Reading data...", input_file)

    # Gera RDDs a partir do arquivo
    rides_rdd = sc.textFile(input_file, use_unicode = True).mapPartitionsWithIndex(cleaning_data).cache()
    
    print("Partition quantity: ", rides_rdd.getNumPartitions())
    return rides_rdd

In [9]:
# To Calculate Top Start Stations
def top_start_stations(num, rides_rdd):
    results = rides_rdd.map(lambda x: (x[3], x[6]-x[5] ) ) \
        .filter(lambda x: x[1].total_seconds() <= 60 * 60 * 2) \
        .mapValues(lambda x:  1 ) \
        .reduceByKey(lambda x,y: x+y) \
        .map(lambda x: (x[1], x[0])) \
        .top(num, key=lambda x: x) 
    return results

In [10]:
# To calculate top routes
def calculate_top_routes(num, rides_rdd): 
    results = rides_rdd.map(lambda x: ( (x[3],x[4]), x[6]-x[5] ) ) \
        .filter(lambda x: x[1].total_seconds() <= 60 * 60 * 2) \
        .mapValues(lambda x: (x.total_seconds(), 1) ) \
        .reduceByKey(lambda x,y: ( x[0] + y[0], x[1] + y[1] ) ) \
        .map(lambda x: (x[1][1], ( x[0], x[1][0]/x[1][1] ) ) ) \
        .top(num, key = lambda x: x)   
    return results

In [27]:
# To calculate stations by gender
def calculate_stations_gender(rides_rdd):
    results = rides_rdd.map(lambda x: (x[1], x[6]-x[5] ) ) \
        .filter(lambda x: x[1].total_seconds() <= 60 * 60 * 2) \
        .mapValues(lambda x: (x.total_seconds(), 1) ) \
        .reduceByKey(lambda x, y: ((x[0] + y[0]), x[1] + y[1]) ) \
        .mapValues(lambda x: (x[0] / x[1], x[1]) ) \
        .collect()
    return results 

In [12]:
# To calculate stations by age
def calculate_stations_age(rides_rdd):
    results = rides_rdd.map(lambda x: (x[2], x[6]-x[5] ) ) \
        .filter(lambda x: x[1].total_seconds() <= 60 * 60 * 2) \
        .mapValues(lambda x: (x.total_seconds(), 1) ) \
        .reduceByKey(lambda x, y: ((x[0] + y[0]), x[1] + y[1]) ) \
        .mapValues(lambda x: (x[0] / x[1], x[1]) ) \
        .collect()
    return results  

In [13]:
# To calculate stations more busy
def calculate_stations_busy(num, rides_rdd, by_count = False):
    results = rides_rdd.map(lambda x: (x[0], x[6]-x[5] ) ) \
        .filter(lambda x: x[1].total_seconds() <= 60 * 60 * 2) \
        .mapValues(lambda x: (1, x.total_seconds())) \
        .reduceByKey(lambda x, y: ((x[0] + y[0]), x[1] + y[1]) ) \
        .map(lambda x: (x[1], x[0])) 
    
    if by_count: 
        return results.top(num, key = lambda x: x[0][0])
    else: 
        return results.top(num, key = lambda x: x[0][1])   

In [23]:
print("\Generating RDD for all bike runs...", file)
rides_rdd = create_rdd(sc, file)

\Generating RDD for all bike runs... dataset.csv
Reading data... dataset.csv
Partition quantity:  2


### 1. Quais são as Top 5 estações com maior número de aluguel de bikes?

In [24]:
top_stations = top_start_stations(5, rides_rdd)
print("Top 5 Estações:\n")
for entry in top_stations:
    print("Estacões: {:03d}, Número de Bikes Alugadas: {:03d}".format(int(entry[1]), entry[0]))

Top 5 Estações:

Estacões: 001, Número de Bikes Alugadas: 6298
Estacões: 027, Número de Bikes Alugadas: 6201
Estacões: 271, Número de Bikes Alugadas: 5262
Estacões: 064, Número de Bikes Alugadas: 4825
Estacões: 041, Número de Bikes Alugadas: 4621


### 2. Quais são as Top 5 rotas, com base na estação inicial e final, e a média de duração de cada aluguel?

In [25]:
top_routes = calculate_top_routes(5, rides_rdd)
print("Top 5 Rotas de Bikes:\n")
for entry in top_routes:
    print("Da Estação: {:03d}, Para a Estação: {:03d}, Número Total de Bikes Alugadas: {:03d}, Duração Média(mins): {:.2f}".format(
        int(entry[1][0][0]), int(entry[1][0][1]), entry[0], entry[1][1]/60))

Top 5 Rotas de Bikes:

Da Estação: 033, Para a Estação: 033, Número Total de Bikes Alugadas: 375, Duração Média(mins): 30.23
Da Estação: 018, Para a Estação: 001, Número Total de Bikes Alugadas: 319, Duração Média(mins): 5.58
Da Estação: 211, Para a Estação: 217, Número Total de Bikes Alugadas: 303, Duração Média(mins): 3.54
Da Estação: 449, Para a Estação: 449, Número Total de Bikes Alugadas: 301, Duração Média(mins): 15.34
Da Estação: 208, Para a Estação: 206, Número Total de Bikes Alugadas: 297, Duração Média(mins): 8.53


### 3. Quem aluga mais bikes, homens ou mulheres? Qual o tempo médio de aluguel de bikes?

In [28]:
gender_stats = calculate_stations_gender(rides_rdd)
print("Perfil de Aluguel de Bikes Por Genero:\n")
for entry in gender_stats:
    print("Genero: {}, Total: {}, Tempo Médio de Aluguel(mins): {:.2f}".format(entry[0], entry[1][1], entry[1][0]/60 ))

Perfil de Aluguel de Bikes Por Genero:

Genero: M, Total: 509782, Tempo Médio de Aluguel(mins): 13.62
Genero: F, Total: 174808, Tempo Médio de Aluguel(mins): 14.30


### 4. Qual faixa etária aluga mais bikes? Qual o tempo médio de aluguel de bikes?

In [32]:
age_stats = calculate_stations_age(rides_rdd)
print("Perfil de Aluguel de Bikes Por Idade:\n")
for entry in age_stats:
    print("Idade: {}, Total: {:06d}, Tempo Médio de Aluguel(mins): {:.2f}".format(entry[0], entry[1][1], entry[1][0]/60 ))

Perfil de Aluguel de Bikes Por Idade:

Idade: 35-44, Total: 167715, Tempo Médio de Aluguel(mins): 13.58
Idade: 18-34, Total: 392040, Tempo Médio de Aluguel(mins): 14.03
Idade: 45-54, Total: 077990, Tempo Médio de Aluguel(mins): 13.30
Idade: 55-64, Total: 035831, Tempo Médio de Aluguel(mins): 13.30
Idade: 00-18, Total: 001332, Tempo Médio de Aluguel(mins): 12.78
Idade: 65+, Total: 009682, Tempo Médio de Aluguel(mins): 13.69


### 5. Quais são as estações com maior número de bikes alugadas/devolvidas?

In [30]:
most_used_bikes = calculate_stations_busy(3, rides_rdd, True)
print("Estações Com Maior Número de Bikes Alugadas/Devolvidas:\n")
for entry in most_used_bikes: 
    print("ID Estação: {:05d}, Total: {}, Minutos: {:.2f}".format(entry[1], entry[0][0], entry[0][1]/60 ))

Estações Com Maior Número de Bikes Alugadas/Devolvidas:

ID Estação: 10771, Total: 217, Minutos: 1523.15
ID Estação: 10810, Total: 208, Minutos: 2785.77
ID Estação: 07854, Total: 192, Minutos: 2633.07


# Fim